### Assignment 14: Assignment: Solving the 4-Queens Problem Using Adversarial Search
Design a competitive version of the 4-Queens problem where two players alternately place queens on a 4x4 chessboard. Implement the Minimax algorithm to strategize queen placements and incorporate alpha-beta pruning for efficient decision-making.

Problem Setup:

The game involves two players (AI and Human) alternately placing queens on a 4x4 chessboard. The goal is to place queens such that no two queens threaten each other. The player unable to make a valid move loses the game.

Rules for Valid Moves:
1. A queen can be placed in any cell of the chessboard, provided it is not attacked by any previously placed queen.
2. A queen attacks cells in the same row, column, and diagonals.
3. Players take turns placing one queen at a time.
4. The game ends when no valid moves are left or all 4 queens are placed.

Game Design:
1. Represent the chessboard as a 4x4 grid.
2. Design functions to check valid placements and update the board.

Minimax Implementation:
1. Use the Minimax algorithm to evaluate all possible moves for both players.
2. Define a utility function:
  1. Positive scores for AI-favorable positions.
  2. Negative scores for opponent-favorable positions.
  3. Zero for neutral or draw positions.

Alpha-Beta Pruning Integration:
1. Optimize the Minimax implementation with alpha-beta pruning.

Player Interaction:
1. Alternate turns between the human player and the AI agent.
2. Display the board after each move for visualization.

Testing and Analysis:
1. Test the AI's performance against a human opponent.
2. Compare execution times for the Minimax algorithm with and without alpha-beta pruning.

In [1]:
import time

BOARD_SIZE = 4

def is_valid(board, r, c):
    for i in range(BOARD_SIZE):
        if board[r][i] == 1 or board[i][c] == 1:
            return False
    for dr, dc in [(-1,-1),(-1,1),(1,-1),(1,1)]:
        i, j = r, c
        while 0 <= i < BOARD_SIZE and 0 <= j < BOARD_SIZE:
            if board[i][j] == 1:
                return False
            i += dr; j += dc
    return True

def get_valid_moves(board):
    moves = []
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            if board[r][c] == 0 and is_valid(board, r, c):
                moves.append((r, c))
    return moves

# Utility: if no moves left => loss for current player
# Return +1 AI win, -1 AI loss, 0 draw (full placement)
def utility(board, ai_turn):
    moves = get_valid_moves(board)
    if not moves:
        return -1 if ai_turn else +1
    if sum(sum(row) for row in board) == BOARD_SIZE:
        return 0
    return None

# Minimax without pruning
def minimax(board, ai_turn):
    val = utility(board, ai_turn)
    if val is not None:
        return val
    moves = get_valid_moves(board)
    if ai_turn:
        best = -float('inf')
        for r, c in moves:
            board[r][c] = 1
            v = minimax(board, False)
            board[r][c] = 0
            best = max(best, v)
        return best
    else:
        best = float('inf')
        for r, c in moves:
            board[r][c] = 1
            v = minimax(board, True)
            board[r][c] = 0
            best = min(best, v)
        return best

# Minimax with alpha-beta pruning
def alphabeta(board, ai_turn, alpha, beta):
    val = utility(board, ai_turn)
    if val is not None:
        return val
    moves = get_valid_moves(board)
    if ai_turn:
        v_best = -float('inf')
        for r, c in moves:
            board[r][c] = 1
            v = alphabeta(board, False, alpha, beta)
            board[r][c] = 0
            v_best = max(v_best, v)
            alpha = max(alpha, v_best)
            if beta <= alpha:
                break
        return v_best
    else:
        v_best = float('inf')
        for r, c in moves:
            board[r][c] = 1
            v = alphabeta(board, True, alpha, beta)
            board[r][c] = 0
            v_best = min(v_best, v)
            beta = min(beta, v_best)
            if beta <= alpha:
                break
        return v_best

# AI chooses move
def pick_move(board):
    best_val = -float('inf')
    best_move = None
    for r, c in get_valid_moves(board):
        board[r][c] = 1
        v = alphabeta(board, False, -float('inf'), float('inf'))
        board[r][c] = 0
        if v > best_val:
            best_val = v
            best_move = (r, c)
    return best_move

def print_board(board):
    for row in board:
        print(' '.join('Q' if x==1 else '.' for x in row))
    print()

board = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
human_turn = True
while True:
    print_board(board)
    if not get_valid_moves(board):
        print(('AI' if human_turn else 'Human') + ' cannot move. ', end='')
        print('Human wins!' if human_turn else 'AI wins!')
        break
    if human_turn:
        move = input('Enter human move as row,col (0-indexed): ')
        r, c = map(int, move.split(','))
        if not is_valid(board, r, c):
            print('Invalid. Try again.')
            continue
        board[r][c] = 1
    else:
        start = time.time()
        r, c = pick_move(board)
        dur = time.time() - start
        print(f'AI moves at {r},{c} (computed in {dur:.4f}s)')
        board[r][c] = 1
    human_turn = not human_turn

# Performance comparison
def test_performance():
    board = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
    # test full tree
    start = time.time(); minimax(board, True); print('Minimax:', time.time()-start)
    start = time.time(); alphabeta(board, True, -float('inf'), float('inf')); print('AlphaBeta:', time.time()-start)

test_performance()

. . . .
. . . .
. . . .
. . . .

Enter human move as row,col (0-indexed): 0,0
Q . . .
. . . .
. . . .
. . . .

AI moves at 1,2 (computed in 0.0002s)
Q . . .
. . Q .
. . . .
. . . .

Enter human move as row,col (0-indexed): 3,1
Q . . .
. . Q .
. . . .
. Q . .

Human cannot move. AI wins!
Minimax: 0.00693511962890625
AlphaBeta: 0.0023148059844970703
